In [19]:
import os
import pandas as pd

DATA_DIR = '/home/jiazhi/Dataset/common-voice_zh-TW_43h_2019-06-12'

In [67]:
def kaldi_gender(gender):
    ''' Alter CommonVoice gender into kaldi format. '''
    if gender == 'male':
        return 'm'
    elif gender == 'female':
        return 'f'
    else:
        return 'm'

In [12]:
# Create directory structures
os.mkdir('data')
os.mkdir('data/train')
os.mkdir('data/test')
os.mkdir('data/local')

In [39]:
# Read tsv files for dataset information
train_tsv = os.path.join(DATA_DIR, 'train.tsv')
test_tsv = os.path.join(DATA_DIR, 'test.tsv')

train_df = pd.read_csv(train_tsv, sep='\t')
test_df = pd.read_csv(test_tsv, sep='\t')
full_df = pd.concat([train_df, test_df])

In [89]:
print(f'train_df.shape: {train_df.shape}')
print(f'test_df.shape:  {test_df.shape}')
train_df.head()

train_df.shape: (1698, 8)
test_df.shape:  (1540, 8)


,client_id,path,sentence,up_votes,down_votes,age,gender,accent
0,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,common_voice_zh-TW_17377831.mp3,我們特別回鄉下,2,0,thirties,male,NaN
1,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,common_voice_zh-TW_17377841.mp3,是歷史上的第二次,2,0,thirties,male,NaN
2,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,common_voice_zh-TW_17377844.mp3,簡單來說,2,1,thirties,male,NaN
3,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,common_voice_zh-TW_17377846.mp3,在田裡也需幫忙,2,0,thirties,male,NaN
4,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,common_voice_zh-TW_17377848.mp3,婚後一年生了個女嬰,2,1,thirties,male,NaN


In [74]:
# Retrieve speaker information for kaldi usage
spk_df = full_df[['client_id', 'gender']].drop_duplicates()
spk_df.gender = spk_df.gender.apply(kaldi_gender)
spk_df['spk_id'] = range(1, 1 + len(spk_df))

In [92]:
print(f'spk_df.shape: {spk_df.shape}')
print(f'spk_df.gender.unique(): {spk_df.gender.unique()}')
spk_df.head()

spk_df.shape: (564, 3)
spk_df.gender.unique(): ['m' 'f']


,client_id,gender,spk_id
0,729aa31a7cb19fbf96fb390f0e7e74dd408a2579ce811a...,m,1
11,8119b4a67d239d11f84bb45ef896a69a94f37563e98213...,m,2
19,811f82987d1a0e11e8d982c27e954766d4cac9fccaeca4...,m,3
26,9bf2140f31463cde58df1859c772e2668a6a81f3c36a88...,f,4
38,14b6e005a4f8fb3a7c89226454caea2dba02acac01604d...,f,5
